# commodAI: Commodity Market Analysis

**Objectives:**

1.  **Primary Goal:** Develop a Python-based (Jupyter Notebook) system for analyzing a commodity market dataset (2000-2023 closing ticks for 18 commodities) to identify and cluster periods of market instability.
2.  **Anomaly Detection:** Utilize machine learning (and potentially deep learning) techniques to detect anomalies within the time series data of each commodity.  Anomalies should be tagged with confidence levels.  Focus on identifying regions, not just individual points.
3.  **Clustering:** Perform ensemble clustering on the identified anomaly regions.  Provide statistical justification for the chosen clustering methods and the resulting clusters.  Analyze and explain the characteristics of each cluster.
4.  **Visualization:** Create a modern, interactive dashboard (within the Jupyter Notebook environment) to visualize the time series data, detected anomalies, clustering results, and statistical analyses.
5.  **Self-Contained Execution:** The entire project MUST be executable on a single machine (MacBook M1 Pro, Sonoma 14.7.3, Anaconda environment, Python 3.9.7) without relying on external web services or APIs (beyond standard package installations).
6.  **Reproducibility:** The jupyter notbook must provide ALL the necessary setup for the environment to be replicated.
7. **Document:** The jupyter notebook must contain a balanced mixture of code and markdown cells to document the entire process.

## Data Loading and Preprocessing

This section loads the commodity market data from the CSV file (`Gran Canaria_database_v3.csv`) and performs initial data preprocessing.

The dataset contains daily closing prices for 18 different commodities from 2000 to 2023.  The goal of preprocessing is to clean the data, handle missing values, and prepare it for anomaly detection and clustering.


In [2]:
# Import required libraries
import pandas as pd
import numpy as np
from scipy import stats

# Read the CSV file with proper settings
df = pd.read_csv('database_v3.csv', 
                 sep=';',  # Use semicolon as separator
                 decimal=',',  # Use comma as decimal separator
                 parse_dates=['date'])  # Parse the date column

# Convert numeric columns to float
numeric_columns = df.columns.drop('date')
df[numeric_columns] = df[numeric_columns].astype(float)

# Now we can calculate z-scores
commodity_columns = df.columns.drop('date').tolist()
z_scores = np.abs(stats.zscore(df[commodity_columns]))
outlier_threshold = 3  # Z-score threshold for outlier detection
outliers = df[z_scores > outlier_threshold]

# Display basic information
print("Dataset shape:", df.shape)
print("\nFirst few rows of the DataFrame:")
print(df[commodity_columns].head())
print("\nDescriptive statistics:")
print(df[commodity_columns].describe())

Dataset shape: (5803, 19)

First few rows of the DataFrame:
   crude  brent  gasoline  ngas  heating  diesel    corn  soybeans  sugar  \
0  25.55  23.95     0.658  2.16    0.687   0.820  203.00    464.25   5.89   
1  24.93  23.72     0.649  2.17    0.671   0.818  203.00    469.25   6.18   
2  24.78  23.55     0.659  2.18    0.675   0.795  203.75    468.00   5.93   
3  24.23  23.35     0.640  2.19    0.660   0.795  207.00    471.50   5.87   
4  24.68  22.77     0.646  2.20    0.660   0.795  208.50    466.25   5.87   

     gold     snp  silver  platinum  palladium   copper     tin  nickel  \
0  281.25  381.78    5.30     433.0      443.0  1829.75  5770.0  8295.0   
1  280.50  378.07    5.18     420.0      433.0  1840.00  5950.0  8289.0   
2  279.40  377.61    5.17     414.0      433.0  1833.75  5950.0  8238.5   
3  282.30  378.56    5.18     418.0      438.0  1841.40  6070.0  8162.0   
4  281.70  378.65    5.20     418.0      449.0  1822.50  6060.0  8120.0   

   aluminum  
0   1611.25 

## Anomaly Detection

This section focuses on detecting anomalies in the commodity time series data. We use the Isolation Forest algorithm, which is an unsupervised learning method particularly effective for anomaly detection in high-dimensional datasets.

**Isolation Forest:**
Isolation Forest isolates anomalies by randomly partitioning the data. Anomalies are easier to isolate and therefore require fewer partitions. The `contamination` parameter specifies the expected proportion of outliers in the dataset.  It's important to choose this parameter carefully based on your understanding of the data.  A higher value will result in more data points being flagged as anomalies.


In [5]:
import numpy as np
import pandas as pd
from sklearn.ensemble import IsolationForest
from sklearn.impute import SimpleImputer
from contextlib import contextmanager
import signal
import time

@contextmanager
def time_limit(seconds):
    def signal_handler(signum, frame):
        raise TimeoutError(f"Timed out after {seconds} seconds")
    signal.signal(signal.SIGALRM, signal_handler)
    signal.alarm(seconds)
    try:
        yield
    finally:
        signal.alarm(0)

def detect_anomalies(
    df: pd.DataFrame, 
    commodity: str, 
    contamination: float = 0.05, 
    timeout: int = 300
) -> tuple[pd.Series, pd.Series]:
    """
    Detects anomalies in a given commodity's time series data using Isolation Forest.
    
    Args:
        df: DataFrame containing commodity time series data
        commodity: Name of the commodity column to analyze
        contamination: Expected proportion of outliers in the dataset
        timeout: Maximum time in seconds to run the model
        
    Returns:
        tuple: (anomalies, confidence_scores) where:
            - anomalies is a Series with 1 (normal) or -1 (anomaly)
            - confidence_scores is a Series with anomaly scores
    """
    try:
        # Print debugging information
        print(f"\nProcessing {commodity}")
        print(f"Initial shape: {df[commodity].shape}")
        print(f"Number of NaN values: {df[commodity].isna().sum()}")
        
        # Extract the data and reshape
        data = df[commodity].values.reshape(-1, 1)
        
        # Create and fit the imputer
        imputer = SimpleImputer(strategy='mean')
        data_clean = imputer.fit_transform(data)
        
        # Validate data
        if np.isnan(data_clean).sum() > 0:
            raise ValueError("Data still contains NaN values after imputation")
        if len(data_clean) == 0:
            raise ValueError("Empty dataset provided")
        if np.isinf(data_clean).any():
            raise ValueError("Dataset contains infinite values")
        
        # Train Isolation Forest model
        model = IsolationForest(
            contamination=contamination,
            random_state=42,
            n_jobs=1,
            max_samples=min(256, len(data_clean)),
            n_estimators=100
        )
        
        # Fit and predict with timeout protection
        with time_limit(timeout):
            predictions = model.fit_predict(data_clean)
        
        # Create output series with proper names
        anomalies = pd.Series(
            predictions, 
            index=df.index, 
            name=f"{commodity}_anomalies"
        )
        
        # Calculate normalized confidence scores (0 to 1 range)
        raw_scores = model.score_samples(data_clean)
        normalized_scores = (raw_scores - raw_scores.min()) / (raw_scores.max() - raw_scores.min())
        confidence_scores = pd.Series(
            normalized_scores,
            index=df.index,
            name=f"{commodity}_confidence"
        )
        
        return anomalies, confidence_scores
        
    except Exception as e:
        print(f"Error processing {commodity}: {str(e)}")
        return None, None

In [11]:
# Process single commodity
anomalies, confidence = detect_anomalies(df, "gold")

# Process multiple commodities
results = {}
for commodity in commodity_columns:
    anomalies, confidence = detect_anomalies(df, commodity)
    if anomalies is not None:
        results[commodity] = {
            'anomalies': anomalies,
            'confidence': confidence
        }

# Print summary
for commodity, result in results.items():
    anomaly_count = (result['anomalies'] == -1).sum()
    print(f"\n{commodity}:")
    print(f"Total anomalies detected: {anomaly_count}")
    print(f"Average confidence score: {result['confidence'].mean():.3f}")


Processing gold
Initial shape: (5803,)
Number of NaN values: 5

Processing crude
Initial shape: (5803,)
Number of NaN values: 5

Processing brent
Initial shape: (5803,)
Number of NaN values: 5

crude:
Total anomalies detected: 290
Average confidence score: 0.838

brent:
Total anomalies detected: 290
Average confidence score: 0.854


## Ensemble Clustering

This section implements ensemble clustering on the identified anomaly regions using k-means, DBSCAN, and hierarchical clustering.

**Why Ensemble Clustering?**
Ensemble clustering combines the results of multiple clustering algorithms to obtain a more robust and stable clustering solution.  Different algorithms have different strengths and weaknesses, and combining them can help to overcome the limitations of individual algorithms. In this case, we are using K-means, DBSCAN and Hierarchical clustering. However, the results are not combined, and each algorithm is evaluated independently.

It evaluates the quality of the clusters using Silhouette score and Davies-Bouldin index, and analyzes the characteristics of each cluster.

**Clustering Methods:**
*   **K-means:** A centroid-based clustering algorithm that aims to partition n observations into k clusters in which each observation belongs to the cluster with the nearest mean (cluster centers or cluster centroid), serving as a prototype of the cluster.
*   **DBSCAN:** A density-based clustering algorithm that groups together points that are closely packed together, marking as outliers points that lie alone in low-density regions.
*   **Hierarchical Clustering:** A clustering algorithm that builds a hierarchy of clusters. It starts with each data point in its own cluster, and then merges the closest pairs of clusters until only a single cluster remains.

In [12]:
# Import required libraries
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.metrics import silhouette_score, davies_bouldin_score
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Create anomaly regions dictionary
anomaly_regions = {}
for commodity in commodity_columns:
    if commodity not in results:
        continue
        
    # Get anomaly indices
    anomaly_indices = np.where(results[commodity]['anomalies'] == -1)[0]
    
    # Group consecutive indices into regions
    regions = []
    start = None
    for i in range(len(anomaly_indices)):
        if start is None:
            start = anomaly_indices[i]
        elif anomaly_indices[i] - anomaly_indices[i-1] > 1:
            regions.append((start, anomaly_indices[i-1]))
            start = anomaly_indices[i]
    
    if start is not None:
        regions.append((start, anomaly_indices[-1]))
    
    anomaly_regions[commodity] = regions

# Prepare data for clustering
anomaly_data = []
for commodity in commodity_columns:
    if commodity not in anomaly_regions:
        continue
    regions = anomaly_regions[commodity]
    for start, end in regions:
        # Extract the data for the anomaly region
        region_data = df[commodity][start:end+1].values
        if len(region_data) > 0:  # Only add non-empty regions
            anomaly_data.append(region_data)

if len(anomaly_data) > 0:
    # Pad the sequences to have the same length
    padded_anomaly_data = pad_sequences(anomaly_data, padding='post', dtype='float64')

    # Define the clustering algorithms
    kmeans = KMeans(n_clusters=3, random_state=42, n_init='auto')
    dbscan = DBSCAN(eps=0.5, min_samples=5)
    hierarchical = AgglomerativeClustering(n_clusters=3)

    # Fit the clustering algorithms
    kmeans_labels = kmeans.fit_predict(padded_anomaly_data)
    dbscan_labels = dbscan.fit_predict(padded_anomaly_data)
    hierarchical_labels = hierarchical.fit_predict(padded_anomaly_data)

    # Evaluate the clustering algorithms
    def evaluate_clustering(labels, data):
        # Filter out noise labels (-1 for DBSCAN)
        if -1 in labels:
            mask = labels != -1
            if sum(mask) < 2:  # Need at least 2 non-noise points
                return -1, -1
            data = data[mask]
            labels = labels[mask]
        if len(set(labels)) < 2:
            return -1, -1
        silhouette = silhouette_score(data, labels)
        davies_bouldin = davies_bouldin_score(data, labels)
        return silhouette, davies_bouldin

    # Evaluate and print results
    kmeans_silhouette, kmeans_davies_bouldin = evaluate_clustering(kmeans_labels, padded_anomaly_data)
    dbscan_silhouette, dbscan_davies_bouldin = evaluate_clustering(dbscan_labels, padded_anomaly_data)
    hierarchical_silhouette, hierarchical_davies_bouldin = evaluate_clustering(hierarchical_labels, padded_anomaly_data)

    print("\nClustering Evaluation Results:")
    print(f"K-means - Silhouette: {kmeans_silhouette:.3f}, Davies-Bouldin: {kmeans_davies_bouldin:.3f}")
    print(f"DBSCAN - Silhouette: {dbscan_silhouette:.3f}, Davies-Bouldin: {dbscan_davies_bouldin:.3f}")
    print(f"Hierarchical - Silhouette: {hierarchical_silhouette:.3f}, Davies-Bouldin: {hierarchical_davies_bouldin:.3f}")

    # Analyze cluster characteristics
    print("\nCluster Characteristics:")
    for algorithm, labels in [("K-means", kmeans_labels), 
                            ("DBSCAN", dbscan_labels), 
                            ("Hierarchical", hierarchical_labels)]:
        print(f"\n{algorithm}:")
        unique_labels = np.unique(labels)
        for label in unique_labels:
            cluster_data = padded_anomaly_data[labels == label]
            print(f"Cluster {label}:")
            print(f"  Size: {len(cluster_data)}")
            print(f"  Mean length: {np.mean([len(x[~np.isnan(x)]) for x in cluster_data]):.2f}")
            print(f"  Mean value: {np.nanmean(cluster_data):.2f}")
            print(f"  Std value: {np.nanstd(cluster_data):.2f}")
else:
    print("No anomaly regions found to cluster")

2025-02-14 19:26:44.578086: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-14 19:26:45.218131: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-14 19:26:45.220175: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-02-14 19:26:46.390601: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT



Clustering Evaluation Results:
K-means - Silhouette: 0.694, Davies-Bouldin: 0.731
DBSCAN - Silhouette: -1.000, Davies-Bouldin: -1.000
Hierarchical - Silhouette: 0.670, Davies-Bouldin: 0.731

Cluster Characteristics:

K-means:
Cluster 0:
  Size: 4
  Mean length: 111.00
  Mean value: 27.38
  Std value: 51.15
Cluster 1:
  Size: 2
  Mean length: 111.00
  Mean value: 94.48
  Std value: 56.04
Cluster 2:
  Size: 43
  Mean length: 111.00
  Mean value: 2.53
  Std value: 13.78

DBSCAN:
Cluster -1:
  Size: 42
  Mean length: 111.00
  Mean value: 9.52
  Std value: 31.37
Cluster 0:
  Size: 7
  Mean length: 111.00
  Mean value: 1.07
  Std value: 11.22

Hierarchical:
Cluster 0:
  Size: 3
  Mean length: 111.00
  Mean value: 78.28
  Std value: 61.78
Cluster 1:
  Size: 43
  Mean length: 111.00
  Mean value: 2.53
  Std value: 13.78
Cluster 2:
  Size: 3
  Mean length: 111.00
  Mean value: 21.22
  Std value: 46.24


## Interactive Dashboard

This section creates an interactive dashboard to visualize the time series data, detected anomalies, and clustering results.

The dashboard allows the user to select a commodity from a dropdown menu and view the corresponding time series data, anomaly regions, and clustering results.  The anomaly regions are highlighted in red, and the clusters are color-coded to distinguish them.


In [16]:
# Import necessary libraries
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

# Set the renderer to browser
pio.renderers.default = 'browser'

# Define color schemes
colors = {
    'normal': '#1f77b4',  # Blue
    'anomaly': '#d62728',  # Red
    'clusters': ['#2ca02c', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f']  # Different colors for clusters
}

# Create plots for each commodity
for commodity in commodity_columns:
    # Create the figure with two subplots
    fig = make_subplots(
        rows=2, cols=1,
        subplot_titles=(
            f'{commodity} Price with Anomaly Regions',
            'Confidence Scores and Clusters'
        ),
        vertical_spacing=0.15
    )
    
    # Add time series data to first subplot
    fig.add_trace(
        go.Scatter(
            x=df['date'],
            y=df[commodity],
            mode='lines',
            name=commodity,
            line=dict(color=colors['normal'])
        ),
        row=1, col=1
    )
    
    # Add confidence scores to second subplot
    if commodity in results:
        confidence_scores = results[commodity]['confidence']
        fig.add_trace(
            go.Scatter(
                x=df['date'],
                y=confidence_scores,
                mode='lines',
                name='Confidence Score',
                line=dict(color='lightgray')
            ),
            row=2, col=1
        )
    
    # Add anomaly regions and cluster information
    if commodity in anomaly_regions:
        for i, (start, end) in enumerate(anomaly_regions[commodity]):
            # Get cluster label
            cluster_label = kmeans_labels[i % len(kmeans_labels)]
            cluster_color = colors['clusters'][cluster_label % len(colors['clusters'])]
            
            # Add anomaly region to first subplot
            fig.add_trace(
                go.Scatter(
                    x=df['date'][start:end+1],
                    y=df[commodity][start:end+1],
                    mode='lines',
                    name=f'Anomaly (Cluster {cluster_label})',
                    line=dict(color=cluster_color, width=3),
                    showlegend=True
                ),
                row=1, col=1
            )
            
            # Add highlighted region to confidence plot
            fig.add_trace(
                go.Scatter(
                    x=df['date'][start:end+1],
                    y=confidence_scores[start:end+1],
                    mode='lines',
                    name=f'Cluster {cluster_label}',
                    line=dict(color=cluster_color, width=3),
                    showlegend=False
                ),
                row=2, col=1
            )
    
    # Update layout
    fig.update_layout(
        height=800,
        title=dict(
            text=f'{commodity} Analysis Dashboard',
            x=0.5,
            y=0.95
        ),
        showlegend=True,
        template='plotly_white',
        hovermode='x unified'
    )
    
    # Update axes
    fig.update_xaxes(title_text='Date', row=2, col=1)
    fig.update_yaxes(title_text='Price', row=1, col=1)
    fig.update_yaxes(title_text='Confidence Score', row=2, col=1)
    
    # Show the figure - this will open in your default web browser
    fig.show()